In [ ]:
pip install requests tldextract pandas beautifulsoup4 selenium urllib3 --quiet

In [ ]:
!apt-get update
!apt install -y chromium-chromedriver
!pip install selenium

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import tldextract

# Aggiungi un User-Agent per simulare una richiesta da un browser reale
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:104.0) Gecko/20100101 Firefox/104.0'
}


def get_all_links(url, domain):
    """
    Raccoglie tutti i link unici presenti in una pagina web (fornita come url),
    filtrando solo quelli che appartengono allo stesso dominio del sito, specificato con domain.
    """
    try:
        # Invia la richiesta HTTP con l'header User-Agent
        print('**ok**')
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        links = set()
        for a_tag in soup.find_all('a', href=True):
            href = a_tag['href']
            full_url = urljoin(url, href)
            parsed_url = urlparse(full_url)
            # Filtra i link che appartengono allo stesso dominio
            if tldextract.extract(parsed_url.netloc).registered_domain == domain:
                links.add(full_url)
        return links
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return set()

def crawl_website(start_url, max_depth=2):
    """
    Esegue il web crawling di un sito, partendo da un'URL di base (start_url) e attraversando
    la struttura del sito fino a un massimo di max_depth livelli di profondità.
    """
    domain = tldextract.extract(start_url).registered_domain
    visited = set()
    to_visit = {start_url}
    next_to_visit = set()

    for depth in range(max_depth):
        while to_visit:
            url = to_visit.pop()
            if url not in visited:
                visited.add(url)
                links = get_all_links(url, domain)
                next_to_visit.update(links - visited)

        to_visit, next_to_visit = next_to_visit, set()

    return visited

# Imposta l'URL iniziale e avvia il crawling
start_url = "https://wiki.u-gov.it/confluence/display/SCAIUS/UG3.2%3A+LEONARDO+UserGuide"
all_links = list(set(crawl_website(start_url, max_depth=3)))

# Filtra i link per rimuovere quelli legati alla conferma dei cookie e mantenere solo le pagine in inglese
links_simone = []
for link in all_links:
    if "#editCookieSettings" not in link:
            links_simone.append(link)

print(f"Link trovati: {links_simone}")

In [ ]:
import re
from bs4 import BeautifulSoup
import requests
import os

def extract_text_with_formatting(url):
    try:
        # Ottieni il contenuto della pagina
        response = requests.get(url)
        response.raise_for_status()  # Solleva un errore HTTP per risposte non riuscite (es. 404)
        soup = BeautifulSoup(response.text, 'html.parser')

        def get_formatted_text(element):
            """
            Recupera il testo con una struttura simile a Markdown
            """
            tag_map = {
                'h1': '# ', 'h2': '## ', 'h3': '### ',
                'h4': '#### ', 'h5': '##### ', 'h6': '###### '
            }

            text = ""
            if element.name in ['p', 'li']:
                text += element.get_text() + "\n"
            elif element.name in ['ul', 'ol']:
                for li in element.find_all('li', recursive=False):
                    text += "* " + get_formatted_text(li)
            elif element.name == 'a':
                text += element.get_text() + f" ({element.get('href')})"
            elif element.name in tag_map:
                text += tag_map[element.name] + element.get_text() + "\n"
            elif element.name == 'table':
                text += get_table_text(element)
            else:
                for child in element.children:
                    if isinstance(child, str):
                        text += child
                    else:
                        text += get_formatted_text(child)
            return text

        def get_table_text(table):
            """
            Estrae il testo da una tabella HTML e lo formatta come Markdown.
            """
            rows = table.find_all('tr')
            col_widths = []

            # Calcola la larghezza massima di ogni colonna
            for row in rows:
                cells = row.find_all(['th', 'td'])
                for i, cell in enumerate(cells):
                    cell_text = cell.get_text().strip()
                    if len(col_widths) <= i:
                        col_widths.append(len(cell_text))
                    else:
                        col_widths[i] = max(col_widths[i], len(cell_text))

            # Crea il testo della tabella con colonne formattate
            table_text = ""
            for row in rows:
                cells = row.find_all(['th', 'td'])
                row_text = "|"
                for i, cell in enumerate(cells):
                    cell_text = cell.get_text().strip()
                    row_text += f" {cell_text.ljust(col_widths[i])} |"
                table_text += row_text + "\n"

                # Aggiungi un separatore dopo l'intestazione
                if row == rows[0]:
                    separator = "|"
                    for width in col_widths:
                        separator += f" {'-' * width} |"
                    table_text += separator + "\n"

            return table_text

        # Estrai il contenuto principale della pagina
        main_content = (
            soup.find('main') or
            soup.find('article') or
            soup.find('body') or
            soup.find('div', {'id': 'content'}) or
            soup.find('div', {'role': 'main'}) or
            soup.find('div', {'class': 'content'}) or
            soup.find('div', {'class': 'main-content'})
        )

        if not main_content:
            print(f"Main content not found for URL: {url}")
            return ""

        # Estrai e pulisci il testo formattato
        formatted_text = get_formatted_text(main_content)
        formatted_text = formatted_text.strip()
        formatted_text = re.sub(r'\n{2,}', '\n\n', formatted_text)  # Limita le linee vuote a due
        formatted_text = re.sub(r'\t+', '', formatted_text)  # Rimuove le tabulazioni

        return formatted_text

    except requests.exceptions.HTTPError as e:
        print(f"HTTPError per {url}: {e}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Errore di connessione per {url}: {e}")
        return None


save_folder = "output_texts"
os.makedirs(save_folder, exist_ok=True)  # Crea la cartella se non esiste

# Processa ogni link
for i, url in enumerate(links_simone):
    # Estrai il testo dal link
    formatted_text = extract_text_with_formatting(url)

    # Se l'URL è valido e il testo è stato estratto correttamente
    if formatted_text:
        # Salva il testo in un file .txt
        file_name = f"file_{i}.txt"
        file_path = os.path.join(save_folder, file_name)

        with open(file_path, "w", encoding="utf-8") as f:
            f.write(formatted_text)

        print(f"Testo salvato in: {file_path}")
    else:
        print(f"Salto il link {url} a causa di un errore.")


In [ ]:
!zip -r output_texts.zip output_texts/

from google.colab import files
files.download("output_texts.zip")